What is the precision of datetime in Python? The [documentation](https://docs.python.org/3.4/library/datetime.html#datetime.datetime.now) says

> Return the current local date and time. If optional argument tz is None or not specified, this is like today(), but, if possible, supplies more precision than can be gotten from going through a time.time() timestamp (for example, this may be possible on platforms supplying the C gettimeofday() function).

It goes on [further](https://docs.python.org/3.4/library/time.html#time.time) to say

> Note that even though the time is always returned as a floating point number, not all systems provide time with a better precision than 1 second.

So, the answer is that it depends.

Let's try to figure out what it looks like on Windows using Python 3.4. For reference:

In [1]:
import sys
print(sys.version)

3.4.3 (v3.4.3:9b73f1c3e601, Feb 24 2015, 22:44:40) [MSC v.1600 64 bit (AMD64)]


Let's create a series of datetime objects that we can analyze to find out how far apart they are:

In [2]:
import datetime

dates = []
for _ in range(10000000):
    dates.append(datetime.datetime.now())

Let's load them into Pandas so we can analyze them:

In [3]:
import pandas as pd

date_series = pd.Series(dates)

In [4]:
date_series.head()

0   2015-06-27 13:06:19.089816
1   2015-06-27 13:06:19.089816
2   2015-06-27 13:06:19.089816
3   2015-06-27 13:06:19.089816
4   2015-06-27 13:06:19.089816
dtype: datetime64[ns]

In [5]:
date_series.describe()

count                       10000000
unique                          9085
top       2015-06-27 13:06:24.066101
freq                            1221
first     2015-06-27 13:06:19.089816
last      2015-06-27 13:06:28.457352
dtype: object

Let's remove all duplicates:

In [6]:
uniq_date_series = date_series.drop_duplicates()
uniq_date_series.describe()

count                           9085
unique                          9085
top       2015-06-27 13:06:25.215166
freq                               1
first     2015-06-27 13:06:19.089816
last      2015-06-27 13:06:28.457352
dtype: object

Now let's figure out the delta between all uniqe dates:

In [7]:
deltas = uniq_date_series - uniq_date_series.shift(1)
deltas.describe()

count                      9084
mean     0 days 00:00:00.001031
std      0 days 00:00:00.000690
min      0 days 00:00:00.001000
25%      0 days 00:00:00.001000
50%      0 days 00:00:00.001000
75%      0 days 00:00:00.001000
max      0 days 00:00:00.033002
dtype: object

And the smallest delta is

In [8]:
deltas.min()

Timedelta('0 days 00:00:00.001000')

This means that the smallest increment between two consecutive dates is **1ms**. So we can not use the datetime on Windows to measure events that occur more frequently than 1ms.

And that is in the best case. This number will vary depending on how many other processes are running and what the Python code does in between two measurements.